# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [63]:
# Write your code below.
%load_ext dotenv
%dotenv ../../01_materials/labs/.env



The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [73]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [74]:
import os
from glob import glob

# Write your code below.
# Get the PRICE_DATA directory from environment variables
price_data_dir = os.getenv("PRICE_DATA")
print("PRICE_DATA directory:", price_data_dir)

# Use glob to find all .parquet files in all subdirectories
# The ** pattern with recursive=True will search all levels of subdirectories for .parquet files
parquet_files = glob(os.path.join(price_data_dir, "**", "*.parquet"), recursive=True)

# Filter to ensure only files, in case directories with .parquet suffixes are included
parquet_files = [f for f in parquet_files if os.path.isfile(f)]

# Check if any files were found
if parquet_files:
    print("Filtered Parquet files:", parquet_files)

    # Load the Parquet files into a Dask DataFrame and set 'ticker' as the index
    dd_px = dd.read_parquet(parquet_files).set_index("ticker")
    print(dd_px.head())
else:
    print("No Parquet files found in the specified directory.")



PRICE_DATA directory: ../../05_src/data/prices/
Filtered Parquet files: ['../../05_src/data/prices\\A\\A_2000.parquet\\part.0.parquet', '../../05_src/data/prices\\A\\A_2001.parquet\\part.0.parquet', '../../05_src/data/prices\\A\\A_2002.parquet\\part.0.parquet', '../../05_src/data/prices\\A\\A_2003.parquet\\part.0.parquet', '../../05_src/data/prices\\A\\A_2004.parquet\\part.0.parquet', '../../05_src/data/prices\\A\\A_2005.parquet\\part.0.parquet', '../../05_src/data/prices\\A\\A_2006.parquet\\part.0.parquet', '../../05_src/data/prices\\A\\A_2007.parquet\\part.0.parquet', '../../05_src/data/prices\\A\\A_2008.parquet\\part.0.parquet', '../../05_src/data/prices\\A\\A_2009.parquet\\part.0.parquet', '../../05_src/data/prices\\A\\A_2010.parquet\\part.0.parquet', '../../05_src/data/prices\\A\\A_2011.parquet\\part.0.parquet', '../../05_src/data/prices\\A\\A_2012.parquet\\part.0.parquet', '../../05_src/data/prices\\A\\A_2013.parquet\\part.0.parquet', '../../05_src/data/prices\\A\\A_2014.parquet\

c:\Users\farim\miniconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\core.py:5517: UserWarning: New index has same name as existing, this is a no-op.
  warnings.warn(


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [79]:
import numpy as np
import dask.dataframe as dd

# Assuming dd_px is the Dask DataFrame with the initial data loaded
# For example:
# dd_px = dd.read_parquet('path_to_parquet_files')

# For each ticker, create a new DataFrame with the required features
dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    # Step 1: Add lags for Close and Adj_Close
    lambda x: x.assign(
        Close_lag=x['Close'].shift(1),
        Adj_Close_lag=x['Adj Close'].shift(1)
    )
).assign(
    # Step 2: Calculate returns based on Adjusted Close
    returns=lambda x: (x['Adj Close'] / x['Adj_Close_lag']) - 1
).assign(
    # Step 3: Calculate hi_lo_range as High - Low
    hi_lo_range=lambda x: x['High'] - x['Low']
))





C:\Users\farim\AppData\Local\Temp\ipykernel_58016\4091257848.py:9: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [81]:
# Write your code below.


dd_feat_reset = dd_feat.reset_index()


pd_feat = dd_feat.compute()
pd_feat['rolling_avg_return'] = pd_feat['returns'].rolling(window=10).mean()
print(pd_feat.head())


             Date     Open     High      Low    Close  Adj Close  Volume  \
ticker                                                                     
GPC    2000-01-03  24.8750  24.8750  23.6875  24.0625  10.805056  602300   
GPC    2000-01-04  24.0000  24.0625  23.5625  23.8125  10.692799  397200   
GPC    2000-01-05  23.8125  24.0000  23.6875  23.6875  10.636664  234900   
GPC    2000-01-06  23.6875  24.0000  23.5000  23.6875  10.636664  298400   
GPC    2000-01-07  23.9375  25.3750  23.9375  25.2500  11.338296  512400   

                        sector     subsector  year  Close_lag  Adj_Close_lag  \
ticker                                                                         
GPC     Consumer Discretionary  Distributors  2000        NaN            NaN   
GPC     Consumer Discretionary  Distributors  2000    24.0625      10.805056   
GPC     Consumer Discretionary  Distributors  2000    23.8125      10.692799   
GPC     Consumer Discretionary  Distributors  2000    23.6875      

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

It wasn’t necessary to convert the Dask DataFrame to a Pandas DataFrame to calculate the 10-day rolling average return; it would actually be more efficient to calculate it directly in Dask. Using Dask for the rolling calculation maintains memory efficiency and enables parallel processing, which is especially beneficial for large datasets that may not fit into RAM. Converting to Pandas requires loading the entire dataset into memory, which can lead to memory issues and reduces processing speed, as Pandas operates on a single core. Therefore, performing the calculation within Dask is generally preferred for handling large data effectively.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.